# CITS4403 - P2P Modelling

This notebook demonstrates a complex P2P model using agent gossip protocols (GNUTELLA-like) and graph-theory.

## Steps to Run
1. Run the first Cell and generate a Graph to use for the current runtime.
2. Run the Initiate Cell to propogate file-pieces.
3. Run the Simulate Cell to decide on the type of Simulation, Scenario and other Agent factors.

In [1]:
import sys, os, pathlib
project_root = pathlib.Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.graph import nxgraph
import src.agent as agent
from utils.plotter import draw_graph, draw_gossip_step_by_step, start_new_run, create_round_gif
#from utils.simulator_old import get_network_stats, reset_simulation
from utils.simulator_new import simulate_round_agent_driven, get_network_stats, reset_simulation
from widgets.graph_widget import display_graph_widgets, get_graph, is_graph_generated
from widgets.init_widget import display_init_widgets, set_graph_data

import ipywidgets as widgets
from IPython.display import display, clear_output

display_graph_widgets()

Use the controls above to configure and generate your graph.
Once you click 'Generate Graph', you can proceed to the next cell.
The graph will be available for use in subsequent cells.


<IPython.core.display.Javascript object>

In [ ]:
# Init
# RUN THE PREVIOUS CELL FIRST

# Check if graph has been generated
if not is_graph_generated():
    print("No Graph")
    print("Run the first cell, click 'Generate Graph' to create a network.")
    raise RuntimeError("Graph must be generated before proceeding")

G, FILE_PIECES = get_graph()

# Set the graph data for the init widget
set_graph_data(G.graph, FILE_PIECES)

print(f"Using graph with {G.graph.number_of_nodes()} nodes and {G.graph.number_of_edges()} edges")

# Display initialization widgets
display_init_widgets()


In [ ]:
# Start simulation with the current graph

if not is_graph_generated():
    print("No Graph")
    print("Run the first cell, click 'Generate Graph' to create a network.")
    raise RuntimeError("Graph must be generated before proceeding")

# Get the graph and file pieces
G, FILE_PIECES = get_graph()

print(f"Starting Model graph with {G.graph.number_of_nodes()} nodes and {G.graph.number_of_edges()} edges")

stats = get_network_stats(G.graph, FILE_PIECES)
print(f"Initial: {stats['incomplete_leechers']} leechers incomplete, {stats['completion_rate']:.1%} completion rate")
output_dir = start_new_run()

retry_stats = {
    'total_retries': 0,
    'pieces_retried': set(),
    'rounds_to_completion': {},
    'previous_failed_pieces': {}
}


# Run one round of simulation
for round_num in range(1, 70):
    print(f"\n Round {round_num}")


#  Simulate Round Setup
    #     G: NetworkX graph
    #     total_pieces: Total number of file pieces
    #     seed: Random seed for reproducibility
    #     single_agent: Force only this agent to search (if specified)
    #     cleanup_completed_queries: Whether to clean up completed queries (helps with clutter, otherwise leftover q/h bounce around until TTL)
    #     search_mode: Search initiation mode
    #     K: Number of neighbors to forward queries to
    #     ttl: Time-to-live for queries (number of hops)
    #     max_searches_per_round: Maximum concurrent searches (for limited mode)
    #     current_round: The current round number (for retry tracking)
    #     neighbor_selection: Neighbor selection method
    
    # search_mode options:
    #- "single": Only one (random) node searches per round
    #- "realistic": Each agent decides independently

    # neighbor_selection options:
    #- "bandwidth": Select k neighbors based on bandwidth weights
    #- "random": Select k neighbors randomly

    max_ttl = 5
    k=3
    search_mode = "realistic"

    result = simulate_round_agent_driven(G.graph, FILE_PIECES, seed=42+round_num, cleanup_completed_queries=True, search_mode="realistic", current_round=round_num, neighbor_selection="random")
    
    print(f"Messages: {result['total_messages']}")
    print(f"Transfers: {result['total_transfers']}")
    if result['new_completions']:
        print(f"New completions: {result['new_completions']}")
        for node in result['new_completions']:
            retry_stats['rounds_to_completion'][node] = round_num
    
    # Track when pieces fail (timeout)
    for node in G.graph.nodes():
        agent = G.graph.nodes[node].get('agent_object')
        if agent and agent.failed_pieces:
            prev_failed = retry_stats['previous_failed_pieces'].get(node, {})
            for piece, fail_count in agent.failed_pieces.items():
                prev_count = prev_failed.get(piece, 0)
                if fail_count > prev_count:
                    new_failures = fail_count - prev_count
                    retry_stats['total_retries'] += new_failures  # Keep same variable name for compatibility
                    retry_stats['pieces_retried'].add(piece)
                    print(f"  Node {node} failed piece {piece} {new_failures} times (total failures: {fail_count})")
            
            # Update previous state
            retry_stats['previous_failed_pieces'][node] = dict(agent.failed_pieces)
    
    stats = get_network_stats(G.graph, FILE_PIECES)
    print(f"Progress: {stats['incomplete_leechers']} leechers incomplete, {stats['completion_rate']:.1%} completion rate")
    
    # Show some transfer details
    if result['transfers']:
        print("transfers:")
        for transfer in result['transfers']:  # Show first handful
            print(f"  Piece {transfer['piece']}: {transfer['from']} to >> {transfer['to']}")
    
    # Show step-by-step gossip
    draw_gossip_step_by_step(G.graph, result['message_rounds'], result['transfers'], 
                            FILE_PIECES, round_num, save_images=False, max_ttl=max_ttl)
                            
    # if stats['completion_rate'] >= 1.0:
    #     print(f"\nAll nodes have all pieces in{round_num} round")
    #     break


print(f"Total failures across all agents: {retry_stats['total_retries']}")
print(f"Unique pieces that failed: {len(retry_stats['pieces_retried'])}")
print(f"Pieces that failed: {sorted(retry_stats['pieces_retried'])}")
if retry_stats['rounds_to_completion']:
    avg_rounds = sum(retry_stats['rounds_to_completion'].values()) / len(retry_stats['rounds_to_completion'])
    print(f"Average rounds to completion: {avg_rounds:.1f}")
    print(f"Completion times: {retry_stats['rounds_to_completion']}")
    
print(f"\nFinal network state:")
for node in G.graph.nodes():
    info = agent.get_agent_info(G.graph, node)
    print(f"Node {node}: {info['role']} - {info['num_pieces']}/{FILE_PIECES} pieces - Complete: {info['is_complete']}")
#
final_stats = get_network_stats(G.graph, FILE_PIECES)


print(f"\nFinal network statistics:")
print(f"Total nodes: {final_stats['total_nodes']}")
print(f"Total edges: {final_stats['total_edges']}")
print(f"Seeders: {final_stats['seeders']}")
print(f"Leechers: {final_stats['leechers']}")
print(f"Complete leechers: {final_stats['complete_leechers']}")
print(f"Incomplete leechers: {final_stats['incomplete_leechers']}")
print(f"Completion rate: {final_stats['completion_rate']:.1%}")
print(f"Total pieces in network: {final_stats['total_pieces_in_network']}")

# Draw the final graph
print(f"\nFinal graph visualization:")
gifs = create_round_gif(duration=300)
draw_graph(G.graph, total_pieces=FILE_PIECES)